In [2]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*
# from state_input_player import*
# from new_verrsion_print import*

# data_full = pd.read_csv('data_test.csv')
# data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
# all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,1.000000,NOT_INVEST,HOSE,8.873960e+12,231283.594397,8.469288e+09,5.072332e+10,8.857091e+05,6.373902e+10,...,6.778475e+11,5.893811e+11,1.097540e+11,3.105095e+11,4.307930e+11,3.586968e+10,1.243129e+10,3.037054e+11,4.984089e+12,-1.218414e+12
1,90,0.000000,AAA,HOSE,4.893114e+12,0.000000,3.926632e+09,5.918750e+10,5.156710e+06,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
2,90,0.000000,AAT,HOSE,8.485598e+11,1384.000000,1.576288e+08,5.314787e+09,1.027330e+06,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
3,90,0.000000,ABS,HOSE,9.451310e+11,0.000000,6.931579e+08,1.141712e+10,7.377700e+05,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
4,90,0.000000,ACC,HOSE,1.911000e+12,258.000000,9.234700e+04,5.818817e+09,3.242300e+05,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,29,0.829231,GIL,HOSE,2.957500e+11,0.000000,1.178573e+07,4.434805e+08,2.435300e+04,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7120,29,0.792308,VSH,HOSE,8.125000e+12,0.000000,0.000000e+00,4.036377e+09,8.040500e+04,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7121,29,0.741068,FPT,HOSE,2.961458e+13,2550.000000,6.714888e+09,1.364417e+10,1.016800e+05,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7122,29,0.724551,GMD,HOSE,7.643971e+12,0.000000,1.093278e+07,9.889580e+09,1.322200e+05,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [7]:
P_NUMBER_COMP_INDEX

964

In [ ]:
# result, per_file = normal_main_level1(player_input, 1,[0])
# result, per_file

NameError: name 'normal_main_level1' is not defined

In [5]:
test_file = pd.read_excel('Hose-2022-Xử-lý.xlsx')
test_file


,STT,FORMULA,Time,CURRENT ASSETS,Cash and cash equivalents,Cash,Cash equivalents,Short Term Financial Investments,Trading securities,Provisions for devaluation of trading securities,...,BUY,SELL,Volume,VolumeARG,ValueARG,MARKET_CAP,Rank Formula,Rank net profit,Sell/Buy,Rank sell/buy
0,1,0.050399,2006,86746430217,4958015179,4958015179,-,24000000000,24000000000,-,...,123000.0,74400.0,3300000,25086,3269100000,4.059000e+11,22,35,0.604878,40
1,2,0.050681,2006,274879281482,12961401109,12961401109,502904000,24521727700,24521727700,-,...,133000.0,60240.0,7887578,18464,2590900000,1.049048e+12,21,24,0.452932,65
2,3,0.018139,2006,156306589247,22569254239,10569254239,12000000000,35000000000,35000000000,-,...,67000.0,96760.0,8990000,79962,5474700000,6.023300e+11,40,40,1.444179,3
3,4,0.004721,2006,28566878379,2364254547,2364254547,-,-,-,-,...,25800.0,17000.0,6840000,122194,3138300000,1.764720e+11,58,67,0.658915,27
4,5,0.047587,2006,329832132342,31648710866,31648710866,-,147000000000,147000000000,-,...,54000.0,40230.0,16200000,249250,15586700000,8.748000e+11,24,23,0.745000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,1934,0.001294,2021,6826580000000,3297060000000,1788884000000,1508176000000,190424000000,-,-,...,34050.0,1000.0,2272318410,3892920,128330900000,7.737244e+13,216,18,0.029369,150
1934,1935,0.030116,2021,1442493728000,1013594184000,651672196000,361921987000,98700000000,-,-,...,46250.0,1000.0,110245596,1013420,46453400000,5.098859e+12,115,60,0.021622,175
1935,1936,0.079839,2021,750821857348,566310617349,36310617349,530000000000,-,-,-,...,15500.0,800.0,79866666,822240,12701700000,1.237933e+12,57,113,0.051613,80
1936,1937,0.294625,2021,215999944000,7416463000,7416463000,-,-,-,-,...,11000.0,1000.0,14299880,151630,1634300000,1.572987e+11,14,210,0.090909,20


In [3]:
ALL_IN4_SYS = np.array([LIST_RANK_NOT_INVEST, np.zeros(ALL_QUARTER), np.zeros(ALL_QUARTER)])
LIST_ALL_COMP_PER_QUARTER = []
for j in range(len(index_test)-1, 0, -1):
    LIST_ALL_COMP_PER_QUARTER.append(index_test[j] - index_test[j-1])
env_state, ALL_IN4_SYS = reset(ALL_IN4_SYS, LIST_ALL_COMP_PER_QUARTER)
env_state[CURRENT_QUARTER_INDEX] =24
player_state = state_to_player(env_state)
player_state


array([  8.,   5.,  28.,  32.,  35.,  17.,  11.,  27.,  14.,  29.,  18.,
        15.,   6.,   3.,   1.,   2.,  26.,  21.,  25.,  37.,  12.,  27.,
        26.,   7.,   8.,  20.,  17.,  43.,  10.,  52.,  18.,  45.,  44.,
        46.,  41.,  35.,   4.,  16.,  42.,  47.,  12.,  43.,   4.,  73.,
        30.,  52.,  64.,  67.,  60.,  40.,  11.,  13.,  63.,  32.,  80.,
        14.,  71.,  35.,  36.,  42.,  10.,  48.,  43.,  35.,  47.,  21.,
        26.,  39.,  50.,  18.,   2.,  31.,  33.,  19.,  56.,  12.,  20.,
        38.,  37.,  15.,  34.,  33.,  38.,   6.,   3.,  26.,  10.,  30.,
        21.,  18.,  36.,  37.,   2.,   4.,   1.,   7.,  20.,  35.,  13.,
         9.,   1.,  39.,  47.,  42.,  13.,   2.,  14.,  43.,  36.,  18.,
        49.,  28.,  12.,  32.,   6.,  37.,  27.,  30.,  19.,  46.,  27.,
        41.,  13.,  33.,  16.,  12.,   6.,  26.,   1.,  46.,  19.,  10.,
         8.,   4.,   2.,   9.,  23.,  47.,  15.,  42.,   8.,   6.,  10.,
        19.,  16.,  17.,  20.,  12.,   2.,  13.,  1

In [6]:
len(player_state)

965

In [46]:
env_state[CURRENT_QUARTER_INDEX] = 25
player_state = state_to_player(env_state)
player_state


array([  4.,  42.,  33.,  16.,  35.,   2.,   5.,  28.,  13.,  24.,  44.,
        40.,  46.,  45.,  38.,  25.,  39.,  18.,  43.,  15.,  10.,  87.,
        24.,  85.,   2.,  53.,  89.,  11.,  27.,  83.,  74.,  72.,   7.,
        73.,  25.,   9.,  56.,  42.,   8.,  21.,   4.,   9.,  65.,  68.,
        67.,  17.,  66.,  41.,   6.,  21.,  34.,  53.,  18.,  69.,  47.,
        28.,   3.,  31.,  19.,   2.,  20.,  16.,   2.,  28.,  35.,   7.,
        37.,  10.,  26.,   3.,  18.,  30.,   6.,  12.,   1.,  36.,  23.,
         4.,  21.,  13.,  21.,  25.,  35.,  27.,  19.,  46.,  30.,  31.,
        50.,  36.,   6.,  43.,  28.,  37.,  13.,  12.,   5.,   2.,  17.,
        16.,  35.,  29.,  23.,  30.,  31.,  36.,  33.,  18.,  15.,  21.,
        14.,   4.,   1.,   9.,  19.,  44.,  38.,  46.,  10.,  26.,  13.,
         8.,  12.,  17.,  20.,  19.,   9.,   4.,  11.,  15.,  10.,   3.,
        16.,   2.,   6.,  18.,  14.,   7.,   5.,  21.,  18.,  34.,  61.,
        29.,  52.,  66.,  46.,  12.,   2.,  54.,  2

In [41]:
env_state[CURRENT_QUARTER_INDEX]

23.0

In [20]:
test1 = [0, [[19, 12, 10, 6, 7, 15, 13, 2, 17, 21, 14], [20, 1, 24, 4, 0, 0, 0, 0, 0, 0, 0]], 1, [[15, 6, 6, 18, 6, 11, 25, 15, 12, 10, 25, 13, 2, 24, 21, 2], [19, 9, 4, 22, 7, 7, 3, 5, 23, 0, 0, 0, 0, 0, 0, 0]], 0, [[6, 24, 4, 16, 2, 3, 2, 16, 9, 8, 16, 22], [21, 18, 10, 21, 18, 0, 0, 0, 0, 0, 0, 0]], 0, [[19, 1, 17, 18, 5, 9, 9, 20, 4, 14, 20, 5], [25, 3, 3, 10, 23, 0, 0, 0, 0, 0, 0, 0]]]



test2 =  [0, [[12, 6, 3], [7, 21, 0]]]

In [21]:




LIST_RANK_NOT_INVEST

array([ 18,  45,   8,   2,  10,   6,   4,  19,  68,  86,  16,  88,  25,
        16,  54,   9,  13,  25,  16,  48,   9,  10,  47,  22,  30,  40,
        81,  48,  55,  68,  29,  29,  67,  48,  37,  74,  92,  78,  50,
        73,  81,  44,  93,  75,  38,  53,  47,  70,  71,  74,  27,  39,
        88, 123, 124, 136, 102, 175, 102,  72,  56,   1], dtype=int64)

In [22]:
def test_ct(test):
    result_fomula = np.zeros(data_arr.shape[1])
    op = -1
    for item in test:
        if type(item) == int:
            if item == 0:
                op = 0
            else:
                op = 1
        else:
            result_temp = np.zeros(data_arr.shape[1])+1
            for id in range(len(item[0])):
                num = data_arr[item[0][id]]
                denom = data_arr[item[1][id]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
            if op == 1:
                result_fomula = result_fomula + result_temp
            elif op == 0:
                result_fomula = result_fomula - result_temp
            else:
                raise Exception('toang cmnr')
            op = -1
    return result_fomula

test_ct1 = test_ct(test2)

In [10]:
test_ct1

array([-2.26622848e+01,  3.13509693e-02, -2.26636784e+01, ...,
       -5.58899903e-22,  6.12750329e-03,  9.17629342e-12])

In [11]:
temp = np.zeros(ALL_QUARTER)

In [12]:
index_test

array([   0,  203,  439,  665,  915, 1107, 1322, 1509, 1672, 1830, 1980,
       2118, 2252, 2398, 2537, 2660, 2784, 2902, 3029, 3188, 3331, 3485,
       3622, 3735, 3856, 3978, 4117, 4221, 4352, 4450, 4546, 4647, 4761,
       4855, 4947, 5045, 5138, 5202, 5268, 5342, 5391, 5448, 5566, 5617,
       5677, 5717, 5785, 5891, 5981, 6123, 6292, 6390, 6542, 6639, 6711,
       6732, 6780, 6831, 6869, 6940, 7029, 7081, 7124])

In [25]:
test_temp = test_ct1[index_test[0]:index_test[1]]
test_temp[:10]
np.argmax(test_temp)

21

In [26]:
np.max(test_temp)

315.98372860844694

In [19]:
test_temp

array([ 2.01994485e-27,  5.76516474e-09,  6.21337306e-02,  6.69152213e-15,
        1.14976623e+24,  4.07042748e-12,  6.62782623e-26, -5.09290673e-04,
       -2.41208120e+02,  3.84033835e+03,  2.19862291e-01,  4.33028290e-10,
       -6.29679564e-02, -8.10885432e+10,  3.09135241e+00,  3.84117112e-01,
        2.81144938e+00, -2.31520918e+01,  4.34129381e-14,  6.57688515e-07,
        1.11094644e-12, -7.14317280e-04,  5.57037425e+09,  5.66380046e+26,
        6.97268221e-30, -1.37605897e-08,  4.87291134e+12,  4.25927955e-02,
        1.33915848e-11,  4.49686047e-10, -3.46213909e-13,  1.94043085e-04,
        7.82801321e+00,  1.44983796e-41,  1.81487747e-03, -8.09625324e-06,
       -6.87656024e-44, -1.03658602e-09, -7.72597784e+15, -5.37837908e-30,
       -5.58899903e-22,  6.12750329e-03,  9.17629342e-12])

In [16]:
test_temp

array([ 2.01994485e-27,  5.76516474e-09,  6.21337306e-02,  6.69152213e-15,
        1.14976623e+24,  4.07042748e-12,  6.62782623e-26, -5.09290673e-04,
       -2.41208120e+02,  3.84033835e+03,  2.19862291e-01,  4.33028290e-10,
       -6.29679564e-02, -8.10885432e+10,  3.09135241e+00,  3.84117112e-01,
        2.81144938e+00, -2.31520918e+01,  4.34129381e-14,  6.57688515e-07,
        1.11094644e-12, -7.14317280e-04,  5.57037425e+09,  5.66380046e+26,
        6.97268221e-30, -1.37605897e-08,  4.87291134e+12,  4.25927955e-02,
        1.33915848e-11,  4.49686047e-10, -3.46213909e-13,  1.94043085e-04,
        7.82801321e+00,  1.44983796e-41,  1.81487747e-03, -8.09625324e-06,
       -6.87656024e-44, -1.03658602e-09, -7.72597784e+15, -5.37837908e-30,
       -5.58899903e-22,  6.12750329e-03,  9.17629342e-12])

In [9]:
index_rank = np.argsort(-test_temp)+1
index_rank

array([171,  16, 153,  93, 114,  95,  44, 189, 190, 145,  35, 106, 143,
       101, 105,  41, 140,  15,  52,  58,  46,  53, 149, 181, 173,  67,
       198,  42, 202,  11,   1,  45,  55, 148,  81,  22,  28,  56, 193,
        29, 119, 129, 108,  78, 137, 150,  71, 184, 135,  26, 186, 147,
        74, 126,  40,  82, 176,  49,  70,  19, 154,  92,  34, 132, 136,
       158, 177,  88, 164,  68, 128,  64, 118, 160,  98,  48,  57, 116,
        43,   9,  63,  69, 159,  87, 152,  75, 166,   6,  30,  50,  38,
       142, 199, 113,  36, 100, 141, 175,  37,  18,  17, 111,  99, 112,
       120,  13,  59,  90, 115, 124,  27, 156, 196, 168,  94, 187,  96,
         2,  20,  83,  80,  12, 192, 138,  60, 163, 161, 134,  73,  84,
       167,  47,  89,   7, 110, 133, 157, 172,  24, 122, 162,  21, 123,
         4, 144,  14,  97, 127,  33, 139, 194,  91, 107,  31,  39, 151,
        54,  72, 178,  86, 179, 183, 104,  25,  23, 185,  65, 117,  77,
       102, 201, 109, 169, 170, 146,  76, 130, 165, 200, 121,  8

In [28]:
index_rank[18]

45

In [14]:
data_full[data_full['TIME'] == 30].reset_index(drop=True)

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,30,2.209184,LAF,HOSE,1.136585e+11,0.00000,7.476612e+07,5.847806e+09,76726.000000,1.847703e+09,...,2.308019e+09,2.083630e+10,5.075545e+09,1.072043e+10,2.264731e+09,3.941893e+09,1.042140e+07,8.882732e+08,2.415655e+11,-1.354865e+10
1,30,2.152778,UNI,HOSE,1.080000e+11,0.00000,0.000000e+00,6.887626e+07,27576.000000,3.032969e+08,...,4.109334e+09,0.000000e+00,0.000000e+00,2.653621e+09,0.000000e+00,6.887626e+07,7.785197e+07,0.000000e+00,7.660344e+10,0.000000e+00
2,30,1.833180,TCT,HOSE,3.468745e+11,11680.00000,0.000000e+00,8.649829e+07,15238.000000,7.963850e+07,...,7.749962e+08,7.238000e+07,1.247959e+09,4.297868e+09,1.944864e+07,8.649829e+07,0.000000e+00,0.000000e+00,7.832726e+09,-5.041614e+10
3,30,1.791667,BMC,HOSE,1.227470e+12,0.00000,0.000000e+00,0.000000e+00,39409.000000,6.876118e+08,...,3.066530e+10,2.146062e+08,1.335059e+09,1.416372e+10,2.475464e+09,0.000000e+00,3.103292e+09,0.000000e+00,1.199934e+10,-9.804690e+09
4,30,1.777778,MHC,HOSE,2.957187e+11,493.00000,0.000000e+00,5.108305e+08,30650.000000,1.638872e+08,...,1.155219e+10,0.000000e+00,0.000000e+00,9.648042e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.101339e+10,0.000000e+00
5,30,1.747253,HTV,HOSE,2.184000e+11,691.00000,0.000000e+00,3.113651e+07,46176.000000,1.064728e+09,...,5.119061e+09,5.593000e+08,1.069175e+09,4.099945e+09,5.375243e+08,2.800000e+07,0.000000e+00,6.351266e+09,1.203350e+10,-2.990221e+10
6,30,1.652174,IMP,HOSE,1.274948e+12,0.00000,1.464202e+08,1.381090e+09,34647.000000,2.248509e+09,...,4.043338e+10,4.757884e+10,8.107041e+09,5.325237e+10,3.404297e+09,8.493330e+07,3.587888e+08,1.739738e+10,8.547047e+10,-6.606188e+10
7,30,1.547445,BBC,HOSE,6.960764e+11,0.00000,3.001039e+08,5.385219e+08,82480.000000,1.913998e+09,...,9.928755e+09,3.417099e+10,1.532275e+09,1.872480e+10,2.534027e+09,4.890467e+08,7.696680e+08,1.593339e+09,6.868149e+10,-1.076289e+11
8,30,1.445783,PAC,HOSE,4.259145e+11,0.00000,4.745189e+10,5.819879e+09,25364.000000,6.206058e+08,...,1.795992e+10,1.325163e+10,2.718382e+09,3.048902e+10,6.648893e+09,1.955577e+09,0.000000e+00,2.000000e+09,1.667165e+11,-1.169227e+11
9,30,1.433871,DMC,HOSE,1.707480e+12,0.00000,1.686106e+09,2.874278e+09,34083.000000,2.477612e+09,...,1.152719e+11,8.021556e+10,3.489041e+08,4.363679e+10,4.637607e+09,1.673867e+09,6.271593e+08,0.000000e+00,1.117198e+11,-3.482055e+10


In [4]:
# result, per_file = normal_main(player_random1, 1000,[0])
result, per_file

(array([503., 497.]), [0])

In [ ]:
result, per_file = normal_main(player_random1,  1,[0])
result, per_file

In [3]:
LIST_RANK_NOT_INVEST_CT1


array([ 2,  2,  3,  3,  3,  3,  2,  2,  3,  2,  6,  6, 10, 10,  7,  8,  8,
        6,  5,  5,  9,  6,  7,  8,  9,  9,  9, 10,  9, 11, 12, 10,  9,  8,
       12, 12, 14, 13, 12, 15, 13, 16, 14, 18, 16, 15, 14, 14, 14, 12, 15,
        9, 12, 11, 11, 15, 17, 16, 17, 21, 19, 19], dtype=int64)

In [24]:
data_full.to_csv('Trang.csv', index=False)

In [4]:
LIST_RANK_NOT_INVEST_CT2

array([ 2,  2,  4,  2,  4,  3,  2,  1,  4,  2,  7,  4, 16, 10,  9, 10,  7,
        6,  2,  3,  9,  9,  9, 11, 14, 12, 16, 10, 14, 16, 19, 15, 16, 16,
       20, 16, 23, 15, 17, 17, 22, 26, 21, 23, 24, 20, 18, 17, 23, 23, 19,
       16, 22, 23, 26, 24, 45, 33, 39, 41, 38, 36], dtype=int64)

In [40]:
import pandas as pd
import math
import numpy as np
import heapq
from scipy.stats.mstats import gmean, hmean
import numba as nb
import random


data_full_load = pd.read_csv('data_test.csv')
data_full_load = data_full_load.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
def get_index_T(data_full):
    list_T = data_full['TIME']
    index_T = [0]
    for i in range(len(list_T)-1):
        if list_T[i] != list_T[i+1]:
            index_T.append(i+1)
    index_T.append(len(list_T))
    return index_T  
index_test = get_index_T(data_full_load)
data_full = data_full_load.copy()
for id in index_test[:-1]:
    s = data_full_load.iloc[id]
    s[1] = 1
    s[2] = 'NOT_INVEST'
    # s[4:] = np.full(len(data_full.columns) - 4, 1)
    s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=0)
    data_full = data_full.append(s)
data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
index_test = np.array(get_index_T(data_full))
data_arr = np.array(data_full[data_full.columns[4:]]).T
NUMBER_VARIABLE = len(data_arr)
all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])
TOP_COMP_PER_QUARTER = 20
NUMBER_QUARTER_HISTORY = 24
ALL_QUARTER = len(np.unique(data_full['TIME']))

def get_variable(data_full):
    global A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z, PROFIT, COMPANY
    list_variable = [[]]*26
    list_column = list(data_full.columns)
    PROFIT = np.array(data_full["PROFIT"])
    COMPANY = np.array(data_full["SYMBOL"])
    for i in range(4, len(list_column)):
        list_variable[i-4] = np.array(data_full[list_column[i]])
    [A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z] = list_variable

def save_data():
    np.save('index_test.npy', index_test)
    np.save('data.npy', data_arr)
    np.save('list_rank_profit_not_invest.npy', LIST_RANK_NOT_INVEST)
    np.save('list_company.npy', COMPANY)

get_variable(data_full)

def get_rank_not_invest():
    list_rank_ko_dau_tu = []
    for j in range(len(index_test)-1, 0, -1):
        # profit_q = PROFIT[index_test[j-1]:index_test[j]]
        COMP = COMPANY[index_test[j-1]:index_test[j]]
        list_rank_ko_dau_tu.append(np.where(COMP == 'NOT_INVEST')[0][0]+1)
    return np.array(list_rank_ko_dau_tu)

LIST_RANK_NOT_INVEST = get_rank_not_invest()
LIST_RANK_NOT_INVEST_TEMP = np.zeros(ALL_QUARTER)
LIST_RANK_CT1 = np.zeros(ALL_QUARTER)
LIST_RANK_NOT_INVEST_CT1 = np.zeros(ALL_QUARTER)
LIST_RANK_CT2 = np.zeros(ALL_QUARTER)
LIST_RANK_NOT_INVEST_CT2 = np.zeros(ALL_QUARTER)
LIST_PROFIT_CT1 = np.zeros(ALL_QUARTER)
LIST_PROFIT_CT2 = np.zeros(ALL_QUARTER)


def get_in4_rank_fomula(result_fomula):
    list_rank = []
    list_com = []
    for j in range(len(index_test)-1, 0, -1):
        COMP = COMPANY[index_test[j-1]:index_test[j]]
        rank_thuc = np.argsort(-result_fomula[index_test[j-1]:index_test[j]]) + 1
        list_rank.append(rank_thuc[0])
        list_com.append(COMP[rank_thuc[0]-1])
    return list_rank, list_com

#Hàm này tính cả list_profit
# def get_in4_fomula(fomula):
#     result_ =  np.nan_to_num(eval(fomula), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
#     list_top_comp = []
#     list_profit = []
#     for j in range(len(index_test)-1, 0, -1):
#         rank_thuc = np.argsort(-result_[index_test[j-1]:index_test[j]]) + 1
#         list_profit.append(PROFIT[index_test[j-1]:index_test[j]][rank_thuc[0]-1])
#         list_top_comp.append(rank_thuc[:TOP_COMP_PER_QUARTER])
#     return np.array(list_top_comp).flatten(), np.array(list_profit)

@nb.njit()
def get_in4_fomula(result_fomula, list_rank_not_invest_temp):
    global LIST_RANK_NOT_INVEST, index_test
    # result_ =  np.nan_to_num(eval(fomula), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
    list_top_comp = np.array([-1])
    list_rank_not_invest_ct = np.array([-1])
    
    for j in range(len(index_test)-1, 0, -1):
        top2 = heapq.nlargest(2,result_fomula[index_test[j-1]:index_test[j]])         #lấy top 2 giá trị lớn nhất
        if top2[0] == top2[1] or np.max(result_fomula[index_test[j-1]:index_test[j]]) == np.min(result_fomula[index_test[j-1]:index_test[j]]):
            # print('toang cong thuc', top2, np.max(result_fomula[index_test[j-1]:index_test[j]]), np.min(result_fomula[index_test[j-1]:index_test[j]]))
            return np.array([-1]), np.array([-1]), 0
        rank_thuc = np.argsort(-result_fomula[index_test[j-1]:index_test[j]]) + 1
        id_not_invest = LIST_RANK_NOT_INVEST[-j] 
        if list_rank_not_invest_ct[0] == -1:
            list_rank_not_invest_ct = np.array([np.where(rank_thuc == id_not_invest)[0][0]+1])
            list_top_comp = rank_thuc[:TOP_COMP_PER_QUARTER]
        else:
            list_rank_not_invest_ct = np.append(list_rank_not_invest_ct, np.where(rank_thuc == id_not_invest)[0][0]+1)
            list_top_comp = np.append(list_top_comp, rank_thuc[:TOP_COMP_PER_QUARTER])
    list_rank_not_invest_temp = list_rank_not_invest_ct
    return list_top_comp, list_rank_not_invest_temp, 1
    # return np.array(list_top_comp).flatten(), 1

IN4_CT1_INDEX = 0
IN4_CT2_INDEX = ALL_QUARTER*TOP_COMP_PER_QUARTER
HISTORY_AGENT_INDEX= IN4_CT2_INDEX + ALL_QUARTER*TOP_COMP_PER_QUARTER
HISTORY_SYS_BOT_INDEX = HISTORY_AGENT_INDEX + ALL_QUARTER
HISTORY_PROFIT_AGENT = HISTORY_SYS_BOT_INDEX + ALL_QUARTER
ID_NOT_INVEST_CT1 = HISTORY_PROFIT_AGENT + ALL_QUARTER
ID_NOT_INVEST_CT2 = ID_NOT_INVEST_CT1 + 1
CURRENT_QUARTER_INDEX = ID_NOT_INVEST_CT2 + 1
ID_ACTION_INDEX = CURRENT_QUARTER_INDEX + 1
CHECK_END_INDEX = ID_ACTION_INDEX + 1

P_IN4_CT1 = 0
P_IN4_CT2 = P_IN4_CT1 + TOP_COMP_PER_QUARTER*NUMBER_QUARTER_HISTORY
P_GMEAN_P1 = P_IN4_CT2 + TOP_COMP_PER_QUARTER*NUMBER_QUARTER_HISTORY
P_GMEAN_P2 = P_GMEAN_P1 + 1
P_ID_NOT_INVEST_CT1 = P_GMEAN_P2 + 1
P_ID_NOT_INVEST_CT2 = P_ID_NOT_INVEST_CT1 + 1

save_data()

def reset():
    global LIST_RANK_CT1, LIST_RANK_CT2, LIST_PROFIT_CT1, LIST_PROFIT_CT2, LIST_RANK_NOT_INVEST_CT1, LIST_RANK_NOT_INVEST_CT2, LIST_RANK_NOT_INVEST_TEMP
    '''
    Hàm này trả ra 2 công thức và list top20 comp qua từng quý của công thức và các thông tin cần thiết khác
    '''
    # list_fomula = []
    count_fomula = 0
    while count_fomula < 2:
        result_fomula = create_fomula(data_arr)
        temp, LIST_RANK_NOT_INVEST_TEMP, check = get_in4_fomula(result_fomula, LIST_RANK_NOT_INVEST_TEMP)
        # print('check getin4', len(temp), check, result_fomula[:10], temp[:10])
        # print('TEMP, ', LIST_RANK_NOT_INVEST_TEMP[:10])
        count_fomula += check
        if count_fomula == 1 and check == 1:
            LIST_RANK_CT1 = temp.copy()
            LIST_RANK_NOT_INVEST_CT1 = LIST_RANK_NOT_INVEST_TEMP.copy()
            # list_fomula.append(fomula)
        elif count_fomula == 2 and check == 1:
            LIST_RANK_CT2 = temp.copy()
            LIST_RANK_NOT_INVEST_CT2 = LIST_RANK_NOT_INVEST_TEMP.copy()
            # list_fomula.append(fomula)
    id_not_invest_ct1 = LIST_RANK_NOT_INVEST_CT1[0]
    id_not_invest_ct2 = LIST_RANK_NOT_INVEST_CT2[0]
    current_quarter = 0
    id_action = 0
    check_end_game = 0
    history_agent = np.zeros(ALL_QUARTER*3)
    # LIST_RANK_CT1, LIST_PROFIT_CT1 = get_in4_fomula(list_fomula[0])
    # LIST_RANK_CT2, LIST_PROFIT_CT2 = get_in4_fomula(list_fomula[1])
    env_state = np.concatenate((LIST_RANK_CT1, LIST_RANK_CT2, history_agent, np.array([id_not_invest_ct1, id_not_invest_ct2, current_quarter, id_action, check_end_game])))
    return env_state

@nb.njit
def state_to_player(env_state):
    '''
    Hàm này trả ra lịch sử kết quả của 2 công thức trong các quý trước đó
    '''
    id_action = env_state[ID_ACTION_INDEX]
    player_state = np.zeros(2*NUMBER_QUARTER_HISTORY*TOP_COMP_PER_QUARTER + 4)
    player_state[P_ID_NOT_INVEST_CT1] = env_state[ID_NOT_INVEST_CT1]
    player_state[P_ID_NOT_INVEST_CT2] = env_state[ID_NOT_INVEST_CT2]
    if env_state[CURRENT_QUARTER_INDEX] != 0:
        history_ct1 = env_state[max(IN4_CT1_INDEX, TOP_COMP_PER_QUARTER*(env_state[CURRENT_QUARTER_INDEX]-24)):int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)]
        history_ct2 = env_state[max(IN4_CT2_INDEX, TOP_COMP_PER_QUARTER*(env_state[CURRENT_QUARTER_INDEX]-24)+IN4_CT2_INDEX):int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)+IN4_CT2_INDEX]
        player_state[P_IN4_CT1:P_IN4_CT1+len(history_ct1)] = history_ct1
        player_state[P_IN4_CT2:P_IN4_CT2+len(history_ct2)] = history_ct2
    agent_history = env_state[HISTORY_AGENT_INDEX : HISTORY_AGENT_INDEX+ALL_QUARTER][:int(env_state[CURRENT_QUARTER_INDEX])]
    sys_bot_history = env_state[HISTORY_SYS_BOT_INDEX : HISTORY_SYS_BOT_INDEX+ALL_QUARTER][:int(env_state[CURRENT_QUARTER_INDEX])]
    if env_state[CHECK_END_INDEX] == 1:
        if id_action == 0:
            player_state[P_GMEAN_P1] = np.exp(np.mean(np.log(agent_history)))
            player_state[P_GMEAN_P2] = np.exp(np.mean(np.log(sys_bot_history)))
        else:
            player_state[P_GMEAN_P1] = np.exp(np.mean(np.log(sys_bot_history)))
            player_state[P_GMEAN_P2] = np.exp(np.mean(np.log(agent_history)))
    return player_state

@nb.njit
def step(action, env_state):
    global LIST_RANK_CT1, LIST_RANK_CT2, LIST_PROFIT_CT1, LIST_PROFIT_CT2, LIST_RANK_NOT_INVEST_CT1, LIST_RANK_NOT_INVEST_CT2
    id_action = env_state[ID_ACTION_INDEX]
    result_quarter = 0
    if action == 0:
        result_quarter = LIST_RANK_NOT_INVEST[int(env_state[CURRENT_QUARTER_INDEX])]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = 1
    elif action == 1:
        result_quarter = env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = LIST_PROFIT_CT1[int(env_state[CURRENT_QUARTER_INDEX])]
    else:
        result_quarter = env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)]
        # env_state[int(HISTORY_PROFIT_AGENT+env_state[CURRENT_QUARTER_INDEX])] = LIST_PROFIT_CT2[int(env_state[CURRENT_QUARTER_INDEX])]
    if result_quarter == 0:
        # print('toang',action,  env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)], np.min(LIST_RANK_CT2))
        raise Exception('toang action')
    rank_3_action = np.array([LIST_RANK_NOT_INVEST[int(env_state[CURRENT_QUARTER_INDEX])], env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER)], env_state[int(env_state[CURRENT_QUARTER_INDEX]*TOP_COMP_PER_QUARTER+IN4_CT2_INDEX)]])
    rank_3_action = np.sort(rank_3_action)
    top_action = np.where(rank_3_action == result_quarter)[0][0] + 1
    # print('quarter', int(env_state[CURRENT_QUARTER_INDEX]),'check', 1/top_action, 'action', action, 'topaction',rank_3_action)
    env_state[int(HISTORY_AGENT_INDEX + ALL_QUARTER*id_action +env_state[CURRENT_QUARTER_INDEX])] = (4-top_action)/3
    if env_state[ID_ACTION_INDEX] == 1:
        env_state[CURRENT_QUARTER_INDEX] += 1  
        #rank giá trị công thức của việc không đầu tư
        if env_state[CURRENT_QUARTER_INDEX] < ALL_QUARTER:
            env_state[ID_NOT_INVEST_CT1] = LIST_RANK_NOT_INVEST_CT1[int(env_state[CURRENT_QUARTER_INDEX])]
            env_state[ID_NOT_INVEST_CT2] = LIST_RANK_NOT_INVEST_CT2[int(env_state[CURRENT_QUARTER_INDEX])]
        env_state[ID_ACTION_INDEX] = 0
    else:
        env_state[ID_ACTION_INDEX] = 1

    return env_state

def action_player(env_state, list_player, temp_file, per_file):
    player_state = state_to_player(env_state)
    current_player = int(env_state[ID_ACTION_INDEX])
    played_move,temp_file,per_file = list_player[current_player](player_state, temp_file, per_file)
    return played_move,temp_file, per_file
    
@nb.njit
def check_winner(env_state):
    agent_history = env_state[HISTORY_AGENT_INDEX : HISTORY_AGENT_INDEX+ALL_QUARTER]
    sys_bot_history = env_state[HISTORY_SYS_BOT_INDEX : HISTORY_SYS_BOT_INDEX+ALL_QUARTER]
    np.exp(np.mean(np.log(sys_bot_history)))
    if np.exp(np.mean(np.log(agent_history))) > np.exp(np.mean(np.log(sys_bot_history))): return 0
    else: return 1

@nb.njit
def check_victory(player_state):
    if not (player_state[P_GMEAN_P1] == player_state[P_GMEAN_P2] and player_state[P_GMEAN_P2] == 0):
        if player_state[P_GMEAN_P1] > player_state[P_GMEAN_P2]: return 1
        else: return 0
    else: return -1

@nb.njit()
def create_fomula(data_arr):
    power = np.random.randint(1, 10)
    operand = np.random.randint(1, 10)
    result_fomula = np.zeros(data_arr.shape[1])
    # ct = []
    for i in range(operand):
        op = np.random.randint(2)
        # ct.append(op)
        numerator = np.random.randint(power, NUMBER_VARIABLE - 1 - power)
        denominator = numerator - power
        numer_var = np.random.randint(1, NUMBER_VARIABLE, numerator)
        result_temp = np.zeros(data_arr.shape[1])+1
        if denominator > 0:
            all_var = np.arange(1,NUMBER_VARIABLE)
            for id in range(len(all_var)):
                if all_var[id] in numer_var:
                    all_var[id] = 0
            all_denom_var = all_var[all_var > 0]
            if len(all_denom_var) < denominator:
                all_denom_var = np.append(all_denom_var, np.random.choice(all_var, denominator - len(all_denom_var)))
            denom_var = np.random.choice(all_denom_var, denominator)
            denom_var = np.append(denom_var, np.zeros(numerator-denominator).astype(np.int64))
            denom_var = denom_var.astype(np.int64)
            # ct.append([numer_var, denom_var])
            for idx in range(len(numer_var)):
                num = data_arr[numer_var[idx]]
                denom = data_arr[denom_var[idx]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
        else:
            denom_var = np.zeros(numerator).astype(np.int64)
            # ct.append([numer_var, denom_var])
            for id in range(len(numer_var)):
                num = data_arr[numer_var[id]]
                denom = data_arr[denom_var[id]]
                denom_zero = np.where(denom == 0)[0]
                denom[denom_zero] = 1
                num[denom_zero] = 1
                result_temp =  result_temp*(num/denom)
        if op == 1:
            result_fomula = result_fomula + result_temp
        else:
            result_fomula = result_fomula - result_temp
    return result_fomula

def create_fomula_old():
    all_char = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    all_char = 'BCDEFGHIJKLMNOPQRSTUVWXYZ'
    power = np.random.randint(1, 10)
    # print('bậc: ', power)
    operand = np.random.randint(1, 10)
    # print('số toán hạng: ', operand)
    list_exp_child = []
    ct = '('
    for i in range(operand):
        ct += random.choice('+-')
        numerator = random.randint(power, 25-power)
        denominator = numerator - power
        numer_char = random.choices(all_char, k = numerator)
        ct += '(' + '*'.join(numer_char) + ')'
        # print(ct, 'mẫu', denominator)
        if denominator > 0:
            all_denom_char = list(set(numer_char)^set(all_char))
            # print('CHECK: ', len(all_denom_char) , denominator)
            if len(all_denom_char) < denominator:
                all_denom_char += random.choices(all_char, k = (denominator - len(all_denom_char)))
            denom_char = random.choices(all_denom_char, k = denominator)
            ct += '/(' + '*'.join(numer_char) + ')'
    ct += ')' + '/A'*power
    return ct

def one_game(list_player, temp_file, per_file):
    env_state = reset()
    count_turn = 0
    while count_turn < ALL_QUARTER*2:
        action, temp_file, per_file = action_player(env_state, list_player, temp_file, per_file)
        env_state = step(action, env_state)
        count_turn += 1
    env_state[CHECK_END_INDEX] = 1
    for id_player in range(len(list_player)):
        action, temp_file, per_file = action_player(env_state,list_player,temp_file, per_file)
        env_state[ID_ACTION_INDEX] = (env_state[ID_ACTION_INDEX] + 1)%len(list_player)
    result = check_winner(env_state)
    return result, per_file

def normal_main(agent_player, times, per_file):
    global data_full, data_arr
    count = np.zeros(2)
    # all_id_fomula = np.arange(len(all_fomula))
    list_player = [agent_player, player_random]
    for van in range(times):
        temp_file = [[0],[0]]
        # shuffle = np.random.choice(all_id_fomula, 2, replace=False)
        # list_fomula = all_fomula[shuffle]
        winner, file_per = one_game(list_player, temp_file, per_file)
        if winner == 0:
            count[0] += 1
        else:
            count[1] += 1
    return count, file_per

def player_random1(player_state, temp_file, per_file):
    list_action = np.array([0,1,2])
    action = int(np.random.choice(list_action))
    # print('check: ', player_state[P_ID_NOT_INVEST_CT1], player_state[P_ID_NOT_INVEST_CT2])
    check = check_victory(player_state)
    return action, temp_file, per_file

def player_random(player_state, temp_file, per_file):
    list_action = np.array([0,1,2])
    action = int(np.random.choice(list_action))
    # check = check_victory(player_state)
    return action, temp_file, per_file




C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
C:\Users\Admin\AppData\Local\Temp\ipykernel_17996\3948151812.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=

In [42]:
result, per_file = normal_main(player_random1, 1000,[0])
result, per_file

(array([523., 477.]), [0])

In [35]:
# LIST_RANK_NOT_INVEST = get_rank_not_invest()
# LIST_RANK_NOT_INVEST_TEMP = np.zeros(ALL_QUARTER)
# LIST_RANK_CT1 = np.zeros(ALL_QUARTER)
# LIST_RANK_NOT_INVEST_CT1 = np.zeros(ALL_QUARTER)
# LIST_RANK_CT2 = np.zeros(ALL_QUARTER)
# LIST_RANK_NOT_INVEST_CT2 = np.zeros(ALL_QUARTER)
# LIST_PROFIT_CT1 = np.zeros(ALL_QUARTER)
# LIST_PROFIT_CT2 = np.zeros(ALL_QUARTER)
LIST_RANK_NOT_INVEST_CT2

array([ 9,  6, 12,  7,  9, 17, 16,  3, 23, 28, 37, 20, 45, 43, 29, 32, 25,
       10, 21, 17, 36, 20, 19, 26, 25, 28, 32, 31, 27, 36, 32, 35, 31, 35,
       41, 36, 44, 37, 37, 43, 38, 51, 41, 50, 34, 42, 40, 45, 41, 52, 59,
       50, 46, 58, 62, 56, 48, 58, 47, 58, 49, 62], dtype=int64)

In [16]:
data_full[data_full['TIME'] == 29].reset_index(drop= True)

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,29,1.711111,BMC,HOSE,7.081560e+11,0.000000,0.000000e+00,0.000000e+00,18710.000000,4.425954e+07,...,3.850054e+10,6.002424e+07,1.062190e+09,1.426318e+10,1.774738e+09,0.000000e+00,1.532980e+09,0.000000e+00,7.134287e+09,-8.930718e+09
1,29,1.519455,DHG,HOSE,2.056000e+12,3651.000000,2.089633e+08,5.538020e+09,13098.000000,3.636580e+08,...,4.098770e+10,1.879467e+10,2.290466e+08,1.200689e+11,2.559756e+10,5.259272e+09,4.127791e+09,5.027947e+10,3.926887e+11,-3.955948e+10
2,29,1.493684,DMC,HOSE,1.118150e+12,911.000000,0.000000e+00,2.099700e+09,11120.000000,8.764751e+08,...,1.513942e+10,7.799700e+10,7.650863e+08,3.334393e+10,4.786260e+09,1.316938e+09,2.857209e+08,0.000000e+00,1.663937e+11,-3.162996e+10
3,29,1.442029,NKD,HOSE,1.159200e+12,0.000000,1.122653e+07,2.207868e+09,29836.000000,2.524407e+09,...,2.732350e+10,0.000000e+00,0.000000e+00,2.725238e+10,0.000000e+00,0.000000e+00,8.824735e+07,0.000000e+00,1.282525e+11,0.000000e+00
4,29,1.352381,IMP,HOSE,8.820000e+11,0.000000,6.285308e+07,3.246889e+08,11620.000000,1.966884e+09,...,2.594947e+10,2.203789e+10,6.468908e+09,4.471379e+10,1.763238e+10,9.599190e+07,6.112551e+08,1.023772e+10,6.829974e+10,-6.254593e+10
5,29,1.320312,HAP,HOSE,3.840161e+11,0.000000,2.037698e+09,6.690929e+08,76888.000000,1.424009e+10,...,2.145582e+10,2.833124e+10,7.046008e+09,5.852910e+09,1.718108e+10,0.000000e+00,2.500597e+07,1.799200e+08,1.065146e+11,-1.136635e+10
6,29,1.300578,KDC,HOSE,5.189997e+12,0.000000,1.522705e+09,6.770757e+09,46753.000000,2.078832e+10,...,4.435903e+10,0.000000e+00,0.000000e+00,5.772461e+10,0.000000e+00,0.000000e+00,3.458402e+08,0.000000e+00,3.306629e+11,-9.554810e+10
7,29,1.236364,BBC,HOSE,4.944500e+11,0.000000,1.816363e+07,1.986796e+08,43951.000000,6.475598e+09,...,3.009596e+10,3.046441e+10,5.807089e+09,2.506554e+10,1.398405e+10,1.254196e+08,5.076773e+08,1.739718e+09,6.585761e+10,-1.053482e+11
8,29,1.202614,HAS,HOSE,1.909998e+11,0.000000,1.312103e+07,1.918014e+08,24778.000000,9.405016e+08,...,4.593618e+09,1.532237e+10,5.200319e+09,6.209712e+09,2.895206e+10,1.918014e+08,0.000000e+00,1.218997e+10,1.360166e+11,-6.020308e+09
9,29,1.200000,ABT,HOSE,3.366000e+11,0.000000,9.559947e+08,6.258958e+08,20966.000000,2.426004e+09,...,5.132152e+09,5.066583e+09,9.638023e+08,1.321703e+10,8.771225e+09,5.507247e+08,0.000000e+00,0.000000e+00,0.000000e+00,-1.849061e+10


In [7]:
import numba as nb
import numpy as np

TEST = np.array([1,2,3])
TEST2 = np.arange(3)
print(TEST, TEST2)

@nb.njit()
def test(test):
    # global TEST, TEST2
    a = np.arange(5)
    test = np.append(test, 2)
    return a, test
x, y = test(TEST)
print(x)
print(TEST)

[1 2 3] [0 1 2]
[0 1 2 3 4]
[1 2 3]


In [4]:
TEST2

array([0, 1, 2])

In [6]:
a = np.array(['abc', 'CDF', 'EFG'])
np.save('test.npy', a )

In [7]:
with open('test.npy', 'rb') as f:
    b = np.load(f)
b 

array(['abc', 'CDF', 'EFG'], dtype='<U3')